# Notebook notes

This notebook

# Import libs

In [9]:
import numpy as np
import matplotlib as plt
from collections import OrderedDict

In [10]:
import torch
from torch import nn

# Config Environment

In [8]:
%matplotlib inline

# if gpu is to be used
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cpu


# Architecture

This is a temporary field, move to the lib after finishing the part

In [11]:
class DQN(nn.Module):
    # this function has to be modified later to account for different 
    # possible architectures we might want to test.
    def __init__(self, input_size, width, height, output_size):
        super(DQN, self).__init__()
        
        assert width >= 2

In [14]:
net = DQN(5, 1, 5, 5)

AssertionError: 

In [ ]:
a = np.array()
task1 =  np.random.choise()